In [ ]:
try:
    import my_gym
except ModuleNotFoundError as e:
    !pip install git+https://github.com/osigaud/my_gym

import os
from typing import Tuple, List

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from mazemdp.toolbox import egreedy, egreedy_loc, softmax, discreteProb
from mazemdp.maze_plotter import show_videos
from mazemdp.mdp import Mdp
from my_gym.envs.maze_mdp import MazeMDPEnv

import gym
import my_gym
from IPython.display import Video

from mazemdp import random_policy

# For visualization
os.environ["VIDEO_FPS"] = "5"
if not os.path.isdir("./videos"):
  os.mkdir("./videos")

# Settings
NB_EPISODES = 50
TIMEOUT = 25
MAZE_SIZE = 10
MAZE_RATIO = 0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/osigaud/my_gym to /tmp/pip-req-build-6c5hugmz
  Running command git clone -q https://github.com/osigaud/my_gym /tmp/pip-req-build-6c5hugmz
     |████████████████████████████████| 1.5 MB 7.8 MB/s 
     |████████████████████████████████| 1.3 MB 24.0 MB/s 
  Created wheel for bbrl-gym: filename=bbrl_gym-0.1.0-py3-none-any.whl size=17222 sha256=29fd2e59ea9e67c1e39433009fc28b5290279c03f8f622326cc0097a710bbf4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-cuq34i97/wheels/0b/2c/67/5df37c12451553731420023ff73d12effcae27a38cc5053550
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=58c439e84af443985dd9722a7df0b6d82a1eeddf0dabcf4bf8db14dc69f07a2e
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for mazemdp: filename=mazemdp-0.1.1-py3-none-any.whl size=14721 sha2

In [ ]:
# Create maze environment
env = gym.make("MazeMDP-v0", kwargs={"width": MAZE_SIZE, "height": MAZE_SIZE, "ratio": MAZE_RATIO})
env.reset()
env.init_draw("The maze")

Output()

In [ ]:
def get_policy_from_q(q: np.ndarray) -> np.ndarray:
    # Outputs a policy given the action values
    return np.argmax(q, axis=1)

def actor_critic(
    mdp: MazeMDPEnv,
    nb_episodes:int,
    timeout:int,
    alpha:float,
    render:bool
) -> Tuple[np.ndarray, List[float]]:
    # Initialize the state-action value function
    # alpha is the learning rate
    q = np.zeros((mdp.nb_states, mdp.action_space.size))
    v = np.zeros(mdp.nb_states)
    # policy = random_policy(mdp)

    q0 = np.zeros((mdp.nb_states, mdp.action_space.size))
    q_list = []

    # Run learning cycle
    mdp.timeout = timeout  # episode length

    if render:
        mdp.init_draw("Actor-Critic")

    for _ in tqdm(range(nb_episodes)):
        # Draw the first state of episode i using a uniform distribution over all the states
        s = mdp.reset(uniform=True)
        done = False
        while not done:
            if render:
                # Show the agent in the maze
                mdp.draw_v_pi(q, q.argmax(axis=1))

            # Draw the max action from q table
            a = np.random.choice([
                i for i in range(len(q[s,:])) if q[s,i] == np.max(q[s,:])
            ])

            # Perform a step of the MDP
            y, r, done, _ = mdp.step(a)

            # Update critic
            delta = r + mdp.gamma * v[y] - v[s]
            v[s] += alpha * delta

            # Update actor
            q[s,a] += alpha * delta
            # if q[s,a] > q[s,policy[s]]:
            #   policy[s] = a

            # Update the agent position
            s = y
        q_list.append(np.linalg.norm(np.maximum(q, q0)))

    if render:
        # Show the final policy
        mdp.current_state = 0
        mdp.draw_v_pi(q, get_policy_from_q(q), title="Actor-Critic")
    return q, q_list

In [ ]:
# Create maze environment
env = gym.make("MazeMDP-v0", kwargs={"width": 5, "height": 5, "ratio": MAZE_RATIO})
env.reset()
# env.init_draw("The maze")

alpha = 0.5
q, q_list = actor_critic(env, nb_episodes=NB_EPISODES, timeout=TIMEOUT, alpha=alpha, render=True)

Output()

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Create maze environment
env = gym.make("MazeMDP-v0", kwargs={"width": 10, "height": 10, "ratio": MAZE_RATIO})
env.reset()
# env.init_draw("The maze")

alpha = 0.5
q, q_list = actor_critic(env, nb_episodes=NB_EPISODES, timeout=TIMEOUT, alpha=alpha, render=True)

Output()

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: ignored